## Cuaderno de creación de lista negra de usuarios

El código definido a continuación tiene como objetivo analizar las publicaciones de la carpeta [`datos`](/datos/), y crear una lista de usuarios que no deberían ser considerados en el análisis de datos.

Se consideraran para esta lista usuarios que sean autores de: 

* Publicaciones hechas para fomentar el cuidado de la salud mental
    * Si bien se pueden encontrar palabras clave que indiquen tendencias suicidas, no necesariamente reflejan problemas mentales en quien las publica
* Publicaciones repetidas (spam o bots)

Para ejecutar las siguientes instrucciones, se empleará el lenguaje de R.

Será de utilidad en el análisis, además, el paquete `tm`, que permite realizar análisis sobre texto.

In [ ]:
# Instalar paquetes necesarios
install.packages(
    c(
        "tm",
        "dplyr",
        "stringr",
        "SnowballC"
    ),
    repos = "http://cran.us.r-project.org"
)

In [1]:
# Se cargan las librerías
library('tm')
library('dplyr')
library('stringr')
library('SnowballC')

Warning message:
"package 'tm' was built under R version 4.2.3"
Loading required package: NLP


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'stringr' was built under R version 4.2.3"
Warning message:
"package 'SnowballC' was built under R version 4.2.3"


In [ ]:
# Se carga el archivo de texto
publicaciones_ahorcarme <- read.csv("./datos/ahorcarme_complete.csv")
publicaciones_colgarme <- read.csv("./datos/colgarme_complete.csv")

In [ ]:
head(publicaciones_ahorcarme)

In [ ]:
# Obtenemos del dataframe las que tengan usuarios que aparezcan más de N veces

obtener_publicaciones <- function(dataframe, pub_minimas) {

    dataframe <- dataframe %>% 
        group_by(User) %>% 
        filter(n() > pub_minimas)

    return(dataframe)
}

In [ ]:
# Filtramos los usuarios que tengan más de 5 publicaciones
publicaciones_ahorcarme_reducidas <- obtener_publicaciones(publicaciones_ahorcarme, 5)
publicaciones_colgarme_reducidas <- obtener_publicaciones(publicaciones_colgarme, 5)

In [ ]:
# Se juntan los vectores que contienen los nombres de los usuarios, para ser unificados en una sola lista negra
# Se le suministrarán varios vectores de usuarios, y se devolverá un vector con los usuarios únicos

juntar_usuarios <- function(listas_usuarios) {

    usuarios <- c()

    for (lista in listas_usuarios) {
        usuarios <- c(usuarios, lista)
    }

    usuarios <- unique(usuarios)

    return(usuarios)
}

In [ ]:
# Creamos la lista de los usuarios que aparecen más de 5 veces en las publicaciones

usuarios_ahorcarme <- publicaciones_ahorcarme_reducidas$User
usuarios_colgarme <- publicaciones_colgarme_reducidas$User

lista_negra <- juntar_usuarios(
    c(
        usuarios_ahorcarme,
        usuarios_colgarme
    )
)

In [ ]:
head(lista_negra)

Ahora, podemos revisar manualmente los tweets de los usuarios que colocamos en la lista negra bajo este criterio, para determinar si realmente sus publicaciones son despreciables para el análisis.

In [ ]:
# Obtenemos los tweets de estos usuarios, para revisarlos manualmente

# Ahorcarme
tweets_lisneg_ahorcarme <- publicaciones_ahorcarme_reducidas %>%
    filter(User %in% lista_negra)

# Colgarme
tweets_lisneg_colgarme <- publicaciones_colgarme_reducidas %>%
    filter(User %in% lista_negra)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
write.table(
    tweets_lisneg_ahorcarme,
    file = "./datos_limpios/tweets_lisneg_ahorcarme.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)

# Colgarme
write.table(
    tweets_lisneg_colgarme,
    file = "./datos_limpios/tweets_lisneg_colgarme.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)


Luego de revisar los tweets, encontré un fragmento que hace alusión a una connotación sexual de la palabra concurrente. Por lo tanto, aquellos usuarios cuyas publicaciones que contienen la palabra "ahorcarme" serán añadidos a la lista negra.

Es importante considerar que, si bien emplean la palabra bajo un concepto sexual, puede no ser la única forma en la que la usan; de hecho, logré encontrar varios perfiles en los que hacen un uso sexual y violento del verbo. De tal manera, de este grupo de usuarios que tergiversan de esta forma la acción, únicamente se seleccionarán aquellos que emplean la palabra exclusivamente en un contexto sexual, es decir, en todos los tweets que les fueron recolectados que les pertenecen.

In [ ]:
# Función que obtiene los tweets de un usuario y determina si todos contienen un texto clave

tweetsSonRepetitivos <- function(dataset, usuario, textoClave) {
    # Obtenemos los tweets del usuario
    tweets <- dataset %>%
        filter(User %in% usuario) %>%
        select(Content)
    
    # Revisamos si todos los tweets contienen el texto clave
    tweets_contienen_texto <- sum(grepl(textoClave, as.vector(tweets$Content), ignore.case = TRUE))
    todosContienen <- tweets_contienen_texto == nrow(tweets)
    
    return (todosContienen)
}

In [ ]:
# Función que toma un dataset y una serie de textos clave, y devuelve los usuarios que tienen todos
# sus tweets con al menos uno de esos textos clave

obtenerUsuariosRepetitivos <- function(dataset, textosClave) {
    # Obtenemos los usuarios
    usuarios <- dataset %>%
        select(User) %>%
        unique()
    
    # Obtenemos los usuarios que tienen todos sus tweets con el texto clave
    usuarios_repetitivos <- c()
    
    for (usuario in usuarios$User) {

        for (textoClave in textosClave) {

            if (tweetsSonRepetitivos(dataset, usuario, textoClave)) {
                usuarios_repetitivos <- c(usuarios_repetitivos, usuario)
                break
            }

        }
    }
    
    return (unique(usuarios_repetitivos))
}

In [ ]:
# Fragmento
frags_frase = c(
    "ahorcame por dios ahorcame hasta el punto de que tus manos"
)

# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados, siempre y cuando todos los tweets del usuario contengan la frase

usuarios_frags_ahorcarme <- obtenerUsuariosRepetitivos(publicaciones_ahorcarme, frags_frase)

Otro son fragmentos de una canción, en los tweets con la palabra clave "colgarme" (que se refieren a una acción como "depender de" o "aprovecharse de").

Los fragmentos son los siguientes:

<blockquote>
    <ul>
        <li> Niña, dame una pestaña de tus ojos para colgarme de amor por ti </li>
        <li> Colgarme de cualquiera que le gusta trasnochar </li>
    </ul>
</blockquote>

In [ ]:
# Fragmentos
frags_frase = c(
    "Niña, dame una pestaña de tus ojos para colgarme de amor por ti",
    "Colgarme de cualquiera que le gusta trasnochar",
)
# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados.

usuarios_frag_colgarme <- obtenerUsuariosRepetitivos(publicaciones_colgarme, frags_frase)

Igualmente, se removerá cualquier uso de la llamada en el contexto de una conversación telefónica.

In [ ]:
raiz_verbo_aparece <- function(texto, raices_a_comparar, lenguage) {

    palabras <- unlist(strsplit(texto, " "))
    raices_palabras <- wordStem(palabras, language = lenguage)
    
    for (raiz_palabra in raices_palabras) {
        
        for (raiz_verbo in raices_a_comparar) {
            
            if ( grepl(raiz_verbo, raiz_palabra) ) {
                return(TRUE)
            }
        }

    }

    return(FALSE)

}

In [ ]:
# Usando el paquete tm, se buscarán los tweets que contengan alguna conjugación de la palabras
# "llamar", "marcar", "responder", "contestar"

# Obtenemos primero las raíces de esos verbos, para saber cuáles raíces buscar
palabras_contexto_telefonico <- c(
    "llamar",
    "marcar",
    "responder",
    "contestar"
)

raices_palabras_contexto_telefonico <- wordStem(palabras_contexto_telefonico, language = "spanish")
print(raices_palabras_contexto_telefonico)

In [ ]:
# Para cada tweet, se buscará si contiene alguna de las raíces de las palabras, y si es así, se agregará el índice del tweet
# a una lista, para ser removido posteriormente.
# No quitamos al usuario completamente porque el uso de la palabra "colgar" en una llamada no le excluye de usarla con
# intenciones autolesivas

tweets_colgarme_limpios <- publicaciones_colgarme %>%
    mutate(
        contexto_telefonico = sapply(Content, raiz_verbo_aparece, raices_palabras_contexto_telefonico, "spanish")
    ) %>%
    # Filtramos los tweets que no usen el verbo para referirse a una llamada
    filter(contexto_telefonico == FALSE) %>%
    # Excluimos la columna que ya no nos sirve
    select(-contexto_telefonico)

In [ ]:
# Agregar los usuarios a la lista de usuarios

lista_negra <- juntar_usuarios(
    c(
        lista_negra,
        usuarios_frags_ahorcarme,
        usuarios_frag_colgarme
    )
)

In [ ]:
writeLines(usuarios_frag_colgarme, "usuarios_frag_colgarme.txt")

In [ ]:
# Escribir la lista negra en un archivo
writeLines(
    lista_negra,
    file = "./datos_limpios/lista_negra.txt",
    row.names = FALSE,
    col.names = FALSE,
    quote = FALSE,
    sep = ","
)


A partir de aquí, una vez que hallamos revisado la lista negra de usuarios, vamos a limpiar los conjuntos de datos para obtener únicamente la información útil.

In [ ]:
# Excluir a estos usuarios de los datasets, pero a partir de la lista negra de usuarios ya revisada

# Leer lista
lista_negra_revisada <- readLines("./datos_limpios/lista_negra_revisada.txt", encoding = "UTF-8")

# Ahorcarme
publicaciones_ahorcarme <- publicaciones_ahorcarme %>%
    filter(!User %in% lista_negra_revisada)

# Colgarme
publicaciones_colgarme <- tweets_colgarme_limpios %>%
    filter(!User %in% lista_negra_revisada)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
write.table(
    publicaciones_ahorcarme,
    file = "./datos_limpios/ahorcarme_filtered.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)

# Colgarme
write.table(
    publicaciones_colgarme,
    file = "./datos_limpios/colgarme_filtered.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)